In [44]:
import pandas as pd
from pathlib import Path

In [74]:
!pip install pandasql

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26799 sha256=7c4d1df04463019510afd4c9c837049a6007a68799b30af44d08092cadbdf81e
  Stored in directory: c:\users\kylec\appdata\local\pip\cache\wheels\15\a1\e7\6f92f295b5272ae5c02365e6b8fa19cb93f16a537090a1cf27
Successfully built pandasql


In [75]:
from pandasql import sqldf

In [102]:
file = Path("Data/cars.csv")
df = pd.read_csv(file,on_bad_lines='warn',encoding='utf-16')

In [103]:
df.head()

,Brand,Model,Year,Status,Mileage,Dealer,Price
0,Mazda,CX-5,2023,New,NaN,NaN,36703.0
1,Kia,Sportage,2023,New,NaN,Classic Kia,28990.0
2,Chevrolet,Camaro,2024,New,NaN,Classic Chevrolet Beaumont,41425.0
3,Ford,Bronco,2023,Used,1551.0,Mike Smith Chrysler Dodge Jeep RAM,58900.0
4,Acura,TLX,2021,Used,30384.0,Mike Smith Nissan,34499.0


In [104]:
df.loc[df['Mileage'].isnull()]

,Brand,Model,Year,Status,Mileage,Dealer,Price
0,Mazda,CX-5,2023,New,NaN,NaN,36703.0
1,Kia,Sportage,2023,New,NaN,Classic Kia,28990.0
2,Chevrolet,Camaro,2024,New,NaN,Classic Chevrolet Beaumont,41425.0
7,BMW,M340,2023,New,NaN,BMW of Beaumont,61715.0
8,Hyundai,Sonata,2023,New,NaN,Hyundai of Silsbee,37945.0
...,...,...,...,...,...,...,...
144862,Chevrolet,Corvette,2023,New,NaN,Chase Chevrolet,122455.0
144863,Chevrolet,Corvette,2023,New,NaN,Chase Chevrolet,125930.0
144864,Jeep,Wrangler,2023,New,NaN,Stockton Dodge Chrysler Jeep RAM,53715.0
144865,Jeep,Wrangler,2023,New,NaN,Stockton Dodge Chrysler Jeep RAM,91450.0


In [105]:
df.loc[df['Status'] == 'New']

,Brand,Model,Year,Status,Mileage,Dealer,Price
0,Mazda,CX-5,2023,New,NaN,NaN,36703.0
1,Kia,Sportage,2023,New,NaN,Classic Kia,28990.0
2,Chevrolet,Camaro,2024,New,NaN,Classic Chevrolet Beaumont,41425.0
7,BMW,M340,2023,New,NaN,BMW of Beaumont,61715.0
8,Hyundai,Sonata,2023,New,NaN,Hyundai of Silsbee,37945.0
...,...,...,...,...,...,...,...
144862,Chevrolet,Corvette,2023,New,NaN,Chase Chevrolet,122455.0
144863,Chevrolet,Corvette,2023,New,NaN,Chase Chevrolet,125930.0
144864,Jeep,Wrangler,2023,New,NaN,Stockton Dodge Chrysler Jeep RAM,53715.0
144865,Jeep,Wrangler,2023,New,NaN,Stockton Dodge Chrysler Jeep RAM,91450.0


In [106]:
#dropping used car with NaN mileage
df.drop(df.loc[(df['Status'] == 'Used') & (df['Mileage'].isnull())].index, inplace=True)

In [107]:
#verify dropping successful
df.loc[(df['Status'] == 'Used') & (df['Mileage'].isnull())]

,Brand,Model,Year,Status,Mileage,Dealer,Price


In [108]:
# replace all NaN milage with zero.
df['Mileage'] = df['Mileage'].fillna(0)

In [109]:
#Dropping Dealer Column
del df["Dealer"]

In [110]:
#Droping rows without a price
df.drop(df.loc[df["Price"].isnull()].index, inplace=True)

In [111]:
df.head()

,Brand,Model,Year,Status,Mileage,Price
0,Mazda,CX-5,2023,New,0.0,36703.0
1,Kia,Sportage,2023,New,0.0,28990.0
2,Chevrolet,Camaro,2024,New,0.0,41425.0
3,Ford,Bronco,2023,Used,1551.0,58900.0
4,Acura,TLX,2021,Used,30384.0,34499.0


In [112]:
#New Dim car age base on current year
current_year = pd.Timestamp.now().year
df['Age'] = current_year - df['Year']

In [113]:
#Creating a Machine Learning dataset with average price on new cars
query = """
SELECT Brand, Model, Year, Status, Mileage, Age, AVG(Price) as AveragePrice FROM df
GROUP BY Brand, Model, Year, Status, Mileage
"""

In [114]:
result = sqldf(query)

In [115]:
print(result)

       Brand Model  Year Status   Mileage  Age  AveragePrice
0      Acura   ILX  2013   Used   86557.0   11  13900.000000
1      Acura   ILX  2014   Used  148266.0   10  10996.000000
2      Acura   ILX  2015   Used   77223.0    9  13650.000000
3      Acura   ILX  2016   Used   56546.0    8  16900.000000
4      Acura   ILX  2016   Used   72301.0    8  16999.000000
...      ...   ...   ...    ...       ...  ...           ...
56522  Volvo  XC90  2023   Used    9623.0    1  52000.000000
56523  Volvo  XC90  2023   Used   10141.0    1  52990.000000
56524  Volvo  XC90  2023   Used   10620.0    1  68995.000000
56525  Volvo  XC90  2023   Used   14346.0    1  51995.000000
56526  Volvo  XC90  2024    New       0.0    0  71733.838384

[56527 rows x 7 columns]


In [116]:
avgPrice_df = pd.DataFrame(result)

In [117]:
avgPrice_df.to_csv("Data/carsAvgPrice.csv")

In [118]:
df.head()

,Brand,Model,Year,Status,Mileage,Price,Age
0,Mazda,CX-5,2023,New,0.0,36703.0,1
1,Kia,Sportage,2023,New,0.0,28990.0,1
2,Chevrolet,Camaro,2024,New,0.0,41425.0,0
3,Ford,Bronco,2023,Used,1551.0,58900.0,1
4,Acura,TLX,2021,Used,30384.0,34499.0,3


In [119]:
df.to_csv("Data/carsUpdate.csv")